## Importing Modules

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema,PydanticOutputParser
from groq import Groq
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableParallel,RunnableBranch, RunnableLambda,RunnableSequence,RunnablePassthrough



import os
from typing import TypedDict,Annotated,Optional,Literal
from pydantic import BaseModel, EmailStr, Field

import warnings
warnings.filterwarnings("ignore")

load_dotenv()

True

In [2]:
model = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.5 ## this is creative parameter
    )

## Runnable Sequence

In [4]:


prompt = PromptTemplate(
    template = "Tell me a joke about {topic}",
    input_variables = ["topic"]
)

parser = StrOutputParser()

chain = RunnableSequence(prompt,model,parser)
result = chain.invoke({"topic":"USA"})
print(result)

Why did the Statue of Liberty get a tan?

Because she was always standing in the sun! 🇺🇸☀️  



In [9]:


prompt_1 = PromptTemplate(
    template = "Tell me a joke about {topic}",
    input_variables = ["topic"]
)

prompt_2 = PromptTemplate(
    template = "Explain the following joke {text}",
    input_variables = ["text"]
)

parser = StrOutputParser()

chain = RunnableSequence(prompt_1,model,parser,prompt_2,model,parser)
result = chain.invoke({"topic":"AI"})
print(result)

The joke plays on the double meaning of "neural network."

* **Literal Meaning:** In poker, a "network" of players might be a group of people who work together to win.  

* **AI Meaning:**  A "neural network" is a type of artificial intelligence system inspired by the structure of the human brain. These networks are very good at learning patterns and making decisions, which is crucial for a good poker player.


The humor comes from the unexpected connection between these two meanings. We initially think the joke is about a group of AI players, but it turns out to be about a single AI using its advanced learning capabilities to excel at the game.  




## Runnable parallel

In [11]:
prompt_tweet = PromptTemplate(
    template= 'Generate a tweet about {topic}',
    input_variables = ["topic"]
)

prompt_linkedin = PromptTemplate(
    template = 'Generate a LinkedIn post about {topic}',
    input_variables = ["topic"]
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    "tweet":RunnableSequence(prompt_tweet,model,parser),
    "LinkedIn":RunnableSequence(prompt_linkedin,model,parser),
})

parallel_chain.invoke({"topic":"AI"})

{'tweet': "AI is moving fast! 🤯 From writing poems to composing music, it's blurring the lines between human & machine. What are your thoughts on the ethical implications of this rapid advancement? #AI #Technology #Future 🤖🧠  \n\n",
 'LinkedIn': '##  Is AI a threat or an opportunity? 🤔 \n\nThe rapid advancements in AI are sparking both excitement and concern.  While some fear job displacement and ethical dilemmas, others see AI as a powerful tool for innovation and progress. \n\n**What are your thoughts?**\n\n✅  Do you see AI as a force for good, driving efficiency and solving complex problems?\n\n❌  Are you worried about the potential negative impacts of AI on society and the workforce?\n\nLet\'s have a constructive conversation about the future of AI and how we can harness its potential responsibly. \n\n#AI #ArtificialIntelligence #FutureofWork #Technology #Innovation #Ethics \n\n\n**Optional additions:**\n\n* **Share a relevant article or news story about AI.**\n* **Ask a specific q

## Runnable Passthrough

In [ ]:
passthrough = RunnablePassthrough()
print(passthrough.invoke({"name":"teja"})) ## what ever is the input it gives the output

{'name': 'teja'}


In [15]:

prompt_1 = PromptTemplate(
    template = "Tell me a joke about {topic}",
    input_variables = ["topic"]
)

prompt_2 = PromptTemplate(
    template = "Explain the following joke {text}",
    input_variables = ["text"]
)

parser = StrOutputParser()

In [18]:
joke_generator_chain = RunnableSequence(prompt_1,model,parser)
joke_generator_chain.invoke({"topic":"AI"})

"Why did the AI cross the road? \n\nTo prove it wasn't chicken! 🐔🤖  \n\nLet me know if you'd like to hear another one! 😄  \n\n"

In [20]:
joke_generator_chain = RunnableSequence(prompt_1,model,parser)
parallel_chain = RunnableParallel({
    "joke":RunnablePassthrough(),
    "explanation":RunnableSequence(prompt_2,model,parser)
    
})

final_chain = RunnableSequence(joke_generator_chain,parallel_chain)
final_chain.invoke({"topic":"AI"})

{'joke': 'Why was the AI so good at poker?\n\nBecause it had a neural network!  🃏🧠  \n\n\nLet me know if you want to hear another one! 😊  \n',
 'explanation': 'The joke plays on the double meaning of "neural network." \n\n* **In AI:** A neural network is a complex computer system designed to mimic the structure and function of the human brain. It\'s used for tasks like learning patterns, making predictions, and even playing games.\n\n* **In poker:**  "Network" can refer to a player\'s ability to read other players and understand their strategies, essentially building a "network" of information about their opponents.\n\nThe joke suggests that the AI was good at poker because it had a powerful neural network (the AI kind), which allowed it to analyze players and make strategic decisions just like a human player with a strong poker network.  \n\n\nLet me know if you\'d like to hear another joke! I\'m always up for a laugh. 😄  \n\n'}

## Runnable Lamda

In [22]:
def word_counter(text):
    return len(text.split())

runnablewordcounter = RunnableLambda(word_counter)
runnablewordcounter.invoke("hey whatsup ??")

3

In [ ]:
joke_generator_chain = RunnableSequence(prompt_1,model,parser)
parallel_chain = RunnableParallel({
    "joke":RunnablePassthrough(),
    "wordcount":RunnableLambda(word_counter)
    
})

final_chain = RunnableSequence(joke_generator_chain,parallel_chain)
final_chain.invoke({"topic":"AI"})

{'joke': "Why was the AI so good at poker? \n\nBecause it had a neural network! 🃏  🤖  \n\n\n\nLet me know if you'd like to hear another one! 😊\n\n",
 'wordcount': 27}

## Runnable Branch

In [24]:
prompt_report = PromptTemplate(
    template = "Write a detailed report on {topic}",
    input_variables = ["topic"]
)

prompt_summary = PromptTemplate(
    template="Summarize the following text \n {text}",
    input_variables = ["text"]
)



In [33]:
report_generation_chain = RunnableSequence(prompt_report,model,parser)
branch_chain = RunnableBranch(
    (lambda x : len(x.split())>100,RunnableSequence(prompt_summary,model,parser)), ## if output is > 500  words we re summarize
    RunnablePassthrough(),
    # RunnableLambda(word_counter)
)

final_chain = RunnableSequence(report_generation_chain,branch_chain)
output = final_chain.invoke({"topic":"AI"})
print(output)
print("--------------")
print(runnablewordcounter.invoke(output))

This comprehensive report provides an overview of Artificial Intelligence (AI).  

**Key points:**

* **Definition:** AI involves developing computer systems capable of performing tasks that usually require human intelligence, like learning, problem-solving, and understanding language.

* **Types:** AI encompasses narrow AI (designed for specific tasks), general AI (hypothetical human-level intelligence), super AI (hypothetical AI surpassing human intelligence), and subfields like machine learning, deep learning, and natural language processing.

* **Applications:** AI is transforming industries like healthcare (diagnosis, drug discovery), finance (fraud detection), transportation (self-driving cars), retail (personalized recommendations), education (personalized learning), and manufacturing (predictive maintenance).

* **Benefits:** AI offers increased efficiency, accuracy, innovation, personalized experiences, and improved safety.

* **Challenges:**  Concerns include bias in algorith